In [16]:
# https://www.geeksforgeeks.org/sparse-matrix-representations-set-3-csr/
struct MyCSR
    m::Int64                  # Number of rows
    n::Int64                  # Number of columns
    rowptr::Vector{Int64}     # Row j is in rowptr[j]:(rowptr[j+1]-1)
    colval::Vector{Int64}     # Col indices of stored values
    nzval::Vector{Float64}    # Stored values, typically nonzeros
    MyCSR(m,n,rowptr,colval,nzval)=new(m,n,rowptr,colval, nzval)
end
# CSR is good! remember me!!!!!!
# Base.getindex(::MyCSR, i::Int64, j::Int64) = 3



In [17]:
import Base.*
function *(M::MyCSR, v::Vector{Float64})
    nw=zeros(length(v))
    for row in 1:M.m
        for i in M.rowptr[row]:M.rowptr[row+1]-1
            col=M.colval[i]
            nw[row]+=v[col]*M.nzval[i]
        end
    end
    return nw
end 

* (generic function with 329 methods)

In [18]:
function Densify(a::MyCSR)
    M=zeros(Float64, a.m, a.n)
    for row in 1:a.m
        for i in a.rowptr[row]:a.rowptr[row+1]-1
            c=a.colval[i]
            M[row, c]=a.nzval[i]
        end
    end
    return M
end

Densify (generic function with 1 method)

In [19]:
using LinearAlgebra;
using Latexify;

In [20]:
dummy = Dict{String, Array{String}}("a"=>["b","c"], "b"=>["a"], "c"=>["a","d"], "d"=>["b"])

Dict{String, Array{String, N} where N} with 4 entries:
  "c" => ["a", "d"]
  "b" => ["a"]
  "a" => ["b", "c"]
  "d" => ["b"]

In [21]:
input=dummy

Dict{String, Array{String, N} where N} with 4 entries:
  "c" => ["a", "d"]
  "b" => ["a"]
  "a" => ["b", "c"]
  "d" => ["b"]

In [22]:
damping=0.85
function get_matrix(exitLinks)
    n=length(exitLinks)
    exitCount=Dict{Int64, Int64}()
    encodeNames=String[]
    decodeNames=Dict{String, Int64}()
    i=1
    for (key, list) in exitLinks
        push!(encodeNames, key)
        push!(decodeNames, key=>i)
        push!(exitCount, i=>length(list))
        i+=1
    end
    
    
    enter=Vector{Vector{Int64}}()
    for _ in 1:n
        push!(enter, Vector{Int64}())
    end
    for (f, list) in exitLinks 
        from=decodeNames[f]
        for t in list 
            to=decodeNames[t]
            push!(enter[to], from)
        end
    end
    
    
    rowptr=ones(Int64, n+1)
    colval=Vector{Int64}()
    nzval=Vector{Float64}()
    i=2
    for to in 1:length(enter) 
        rowptr[i]=rowptr[i-1]+length(enter[to])
        for from in enter[to]
            append!(nzval, 1/exitCount[from])
            append!(colval, from)
        end
        i+=1
    end
    return (MyCSR(n,n, rowptr, colval, nzval), encodeNames, n)
end

get_matrix (generic function with 1 method)

In [26]:
function sparse_power(input, k, β=0.85)
    M, encode, n = get_matrix(input)
    v = ones(n)/n
    for i in 1:k
        v=β*(M*v).+(1-β)/n
        v/=sum(v)
    end
    v=map(r-> (encode[r[1]],Float64(r[2])), enumerate(v))
    return sort(v, by=x->x[2], rev=true)
end

sparse_power (generic function with 2 methods)

In [27]:
x=sparse_power(input,100)

4-element Vector{Tuple{String, Float64}}:
 ("a", 0.37824245632609843)
 ("b", 0.3017469560614081)
 ("c", 0.19825304393859183)
 ("d", 0.12175754367390153)